In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

In [3]:
spark = SparkSession.builder \
    .appName("SmallTestDataset") \
    .master("local[*]") \
    .getOrCreate()

In [4]:
#INPUT DATA
input_data_g = "/home/jupyter/proyect/ds_taxi_NY/green_tripdata_2024-01.parquet"
input_data_y = "/home/jupyter/proyect/ds_taxi_NY/yellow_tripdata_2024-01.parquet"

In [5]:
#FUNCIONES
def sumar_missing_per_var(x):
    return x.select([F.count( F.when(F.col(c).isNull(),c)  ).alias(c) for c in x.columns]).show()

In [6]:
#READ FILE GREEN
df_g = spark.read.parquet(input_data_g)

In [17]:
dft= df_g.select( ['lpep_pickup_datetime','lpep_dropoff_datetime'])\
         .withColumn('duracion_s', F.unix_timestamp("lpep_dropoff_datetime") - F.unix_timestamp('lpep_pickup_datetime') )\
          

dft.show()
        

+--------------------+---------------------+----------+
|lpep_pickup_datetime|lpep_dropoff_datetime|duracion_s|
+--------------------+---------------------+----------+
| 2024-01-01 00:46:55|  2024-01-01 00:58:25|       690|
| 2024-01-01 00:31:42|  2024-01-01 00:52:34|      1252|
| 2024-01-01 00:30:21|  2024-01-01 00:49:23|      1142|
| 2024-01-01 00:30:20|  2024-01-01 00:42:12|       712|
| 2024-01-01 00:32:38|  2024-01-01 00:43:37|       659|
| 2024-01-01 00:43:41|  2024-01-01 01:00:23|      1002|
| 2024-01-01 00:31:56|  2024-01-01 00:48:09|       973|
| 2024-01-01 00:46:12|  2024-01-01 00:57:39|       687|
| 2024-01-01 00:38:07|  2024-01-01 00:39:23|        76|
| 2024-01-01 00:44:24|  2024-01-01 00:57:47|       803|
| 2024-01-01 00:18:37|  2024-01-01 00:35:16|       999|
| 2024-01-01 00:47:36|  2024-01-01 01:07:34|      1198|
| 2024-01-01 00:03:57|  2024-01-01 00:18:06|       849|
| 2024-01-01 00:39:58|  2024-01-01 00:58:09|      1091|
| 2024-01-01 00:18:52|  2024-01-01 00:25:10|    

In [24]:
variable = 'duracion_s'
dft.agg(
    F.min(F.col(variable) ),
    F.max(F.col(variable) ),
    F.mean(F.col(variable) )
        )\
        .show()

+---------------+---------------+------------------+
|min(duracion_s)|max(duracion_s)|   avg(duracion_s)|
+---------------+---------------+------------------+
|              0|          86339|1090.2827182543192|
+---------------+---------------+------------------+



In [ ]:
#READ FILE GREEN
df_g = spark.read.parquet(input_data_g)

#SELECCION VARIABLES GREEN DATASET
lista_vars = ['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfg = df_g.select(lista_vars)


# FEATURE duracion_s: duracion del viaje en segundos
dfg = dfg.withColumn('duracion_s', F.unix_timestamp("lpep_dropoff_datetime") - F.unix_timestamp('lpep_pickup_datetime') )\
         .withColumn('duracion_s', F.when(  F.col('duracion_s')<0 , F.col('duracion_s')*-1 ).otherwise(F.col('duracion_s')) )

# TRANSFORMACION REDONDEO TIEMPO, Y DATE GREEN TAXI
dfg = dfg.withColumn("date_init_trip", F.date_format(F.col("lpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("lpep_pickup_datetime") ))\
         .drop(F.col('lpep_pickup_datetime'))\
         .drop( F.col('lpep_dropoff_datetime') )\
         .drop( "date_arrival_trip","hour_arrival_trip"  )

# FEATURE tipoVehiculo: tipo vehiculo
dfg = dfg.withColumn('tipoVehiculo', F.lit('Green'))

In [39]:
dfg.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duracion_s: long (nullable = true)
 |-- date_init_trip: string (nullable = true)
 |-- hour_init_trip: integer (nullable = true)
 |-- tipoVehiculo: string (nullable = false)



In [1]:
df_g.count()

NameError: name 'df_g' is not defined

In [29]:
#READ YELLOW
df_y = spark.read.parquet(input_data_y)
dfty= df_y.select( ['tpep_pickup_datetime','tpep_dropoff_datetime'])\
         .withColumn('duracion_s', F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp('tpep_pickup_datetime') )\
         .withColumn('duracion_s', F.when(  F.col('duracion_s')<0 , F.col('duracion_s')*-1 ).otherwise(F.col('duracion_s')) )
          

# dfty.show()
        
variable = 'duracion_s'
dfty.agg(
    F.min(F.col(variable) ),
    F.max(F.col(variable) ),
    F.mean(F.col(variable) )
        )\
        .show()

+---------------+---------------+-----------------+
|min(duracion_s)|max(duracion_s)|  avg(duracion_s)|
+---------------+---------------+-----------------+
|              0|         567324|936.7798354867261|
+---------------+---------------+-----------------+



In [27]:
dfty.count()

2964624

In [26]:
dfty.filter(F.col('duracion_s')<0).count()

56

In [30]:
dfty.filter(F.col('duracion_s')<0).show()

+--------------------+---------------------+----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|duracion_s|
+--------------------+---------------------+----------+
+--------------------+---------------------+----------+



In [ ]:
#READ YELLOW
df_y = spark.read.parquet(input_data_y)

#SELECCION VARIABLES YELLOW DATASET
lista_vars = ['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'tip_amount',
 'total_amount']

dfy = df_y.select(lista_vars)

# FEATURE duracion_s: duracion del viaje en segundos
dfy = dfy.withColumn('duracion_s', F.unix_timestamp("tpep_dropoff_datetime") - F.unix_timestamp('tpep_pickup_datetime') )\
         .withColumn('duracion_s', F.when(  F.col('duracion_s')<0 , F.col('duracion_s')*-1 ).otherwise(F.col('duracion_s')) )

# TRANSFORMACION REDONDEO TIEMPO, Y DATE YELLOW TAXI
dfy = dfy.withColumn("date_init_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         .withColumn( 'hour_init_trip',     F.hour( F.col("tpep_dropoff_datetime") ))\
         .drop(F.col('tpep_pickup_datetime'))\
         .drop( F.col('tpep_dropoff_datetime') )\
         .drop( "date_arrival_trip","hour_arrival_trip"  )

# FEATURE tipoVehiculo: tipo vehiculo
dfy = dfy.withColumn('tipoVehiculo', F.lit('Yellow'))

In [37]:
dfy.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- duracion_s: long (nullable = true)
 |-- date_init_trip: string (nullable = true)
 |-- hour_init_trip: integer (nullable = true)
 |-- tipoVehiculo: string (nullable = false)



In [ ]:
         #.withColumn("date_arrival_trip", F.date_format(F.col("tpep_pickup_datetime"), "yyyy-MM-dd"))\
         #.withColumn( 'hour_arrival_trip',     F.hour( F.col("tpep_dropoff_datetime") )) \

In [15]:
dfy.columns == dfg.columns

True

In [ ]:
# Union data sets
df = dfg.union(dfy)

In [24]:
# Orden columnas
df = df.select(['VendorID',
            'tipoVehiculo',
            'date_init_trip',
            'hour_init_trip',
            'date_arrival_trip',
            'hour_arrival_trip',              
            'PULocationID',
            'DOLocationID',
            'passenger_count',
            'trip_distance',
            'tip_amount',
            'total_amount'])

# Incluir propinas en el precio total
df = df.withColumn('total_amount', F.round(F.col('total_amount')+F.col('tip_amount'),2)  )

In [25]:
df.show()

+--------+------------+--------------+--------------+-----------------+-----------------+------------+------------+---------------+-------------+----------+------------+
|VendorID|tipoVehiculo|date_init_trip|hour_init_trip|date_arrival_trip|hour_arrival_trip|PULocationID|DOLocationID|passenger_count|trip_distance|tip_amount|total_amount|
+--------+------------+--------------+--------------+-----------------+-----------------+------------+------------+---------------+-------------+----------+------------+
|       2|       Green|    2024-01-01|             0|       2024-01-01|                0|         236|         239|              1|         1.98|      3.61|       28.88|
|       2|       Green|    2024-01-01|             0|       2024-01-01|                0|          65|         170|              5|         6.54|      7.11|       56.88|
|       2|       Green|    2024-01-01|             0|       2024-01-01|                0|          74|         262|              1|         3.08|     

In [ ]:
# CREACION DATAMARTS

df_day=